# Functional Usage

In [1]:
from braandket_circuit import CNOT, H, M, QOperation, Sequential, allocate_qubits

As we already know, a circuit can be defined using `Sequential`, `.on()` and some operations.

In [2]:
circuit = Sequential([
    H.on(0),
    CNOT.on(0, 1),
    M
])


If you feel annoying pointing out target qubit using `.on()` and indices, do try functional usage!

In functional usage, we can directly call `QOperation` instances like functions with qubits as arguments. In most cases, functional usage is more convenient and readable.

Code below performs the whole circuit directly, without `Sequential` and `.on()`. As we can see, calling `M` returns a `MeasurementResult` instance.


In [3]:
q0, q1 = allocate_qubits(2)

H(q0)
CNOT(q0, q1)
result = M(q0, q1)

print(f"{result.value=}")
print(f"{result.prob=}")

result.value=array([0, 0], dtype=int64)
result.prob=array(0.5)


You may want to wrap the circuit up, distinguishing it from other parts. Then you'll find that the circuit can be defined as a function (That's why it is called "functional").

In [4]:
# noinspection PyRedeclaration
def circuit(q0, q1):
    H(q0)
    CNOT(q0, q1)
    return M(q0, q1)


q0, q1 = allocate_qubits(2)
result = circuit(q0, q1)

print(f"{result.value=}")
print(f"{result.prob=}")

result.value=array([1, 1], dtype=int64)
result.prob=array(0.5)


Furthermore, you can define your circuit as a subclass of `QOpeartion` by overriding the `__call__` method.

In [5]:
class MyCircuit(QOperation):
    def __call__(self, q0, q1):
        H(q0)
        CNOT(q0, q1)
        return M(q0, q1)


q0, q1 = allocate_qubits(2)

circuit = MyCircuit()
result = circuit(q0, q1)

print(f"{result.value=}")
print(f"{result.prob=}")

result.value=array([1, 1], dtype=int64)
result.prob=array(0.5)


Defining circuit by subclassing is generally a more complicated way than writing a plain function.

Here are the key differences between them:

* Instances `QOperation` can have commonly used methods and properties, like `.on()` and `name`.
* Instances of `QOperation` can be recorded for visualization, compilation, optimization, etc. 
* You can define additional methods and properties in the subclass of `QOperation`.
* (Advanced) You can define a runtime-dependent implementation for a subclass of `QOperation` via the "apply impl registry" mechanism.